# Урок 3. Домашнее задание

In [49]:
import numpy as np
import matplotlib.pyplot as plt

In [50]:
%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

In [51]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).

In [52]:
# Первоначальная функция:
# def calc_logloss(y, y_pred):
#     err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
#     return err

epsilon = 1e-5
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred + epsilon) - (1.0 - y) * np.log(1.0 - y_pred + epsilon))
    return err

In [53]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [54]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss не превышал значение `0.3` `(0.1).` Как изменились веса?

In [55]:
def eval_LR_model(X, y, iterations, alpha):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [56]:
# w = eval_LR_model(X_st, y, 2000, 1e-1)

w = eval_LR_model(X_st, y, 10000, 1e-1)  # err <= 0.3

1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40586169307445663
2000 [-4.59094777 -1.0359998  -0.13825631  4.44501211] 0.3533237624911082
3000 [-5.91004767 -1.08773886 -0.62127728  5.36155604] 0.32499851416975567
4000 [-6.96133333 -1.14371437 -0.98694403  6.13057114] 0.3066095638641659
5000 [-7.85498033 -1.19935714 -1.28748713  6.80564285] 0.29311009630153567
6000 [-8.64731257 -1.25347799 -1.54786532  7.41682354] 0.2823791730492525
7000 [-9.36966717 -1.30586395 -1.78130777  7.98193089] 0.27338982900832914
8000 [-10.04075209  -1.35657036  -1.99546097   8.51208184] 0.26558808403656786
9000 [-10.67239108  -1.40571631  -2.1950503    9.01453862] 0.2586491627848389
10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.25236965480682827


In [57]:
# w = eval_LR_model(X_st, y, 300000, 1e-1)  # err <= 0.1

3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred_proba`).

In [58]:
def calc_pred_proba(w, X_st):
    y_pred_proba = sigmoid(np.dot(X_st, w))
    return y_pred_proba

In [59]:
calc_pred_proba(w, X_st)

array([0.2896499 , 0.13238445, 0.99730871, 0.19963747, 0.73522164,
       0.28534702, 0.99936607, 0.08615041, 0.34268615, 0.99086395])

4. Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, а также порог вероятности, на выходе - массив `y_pred`).

In [60]:
def calc_pred(w, X_st):
    y_pred = sigmoid(np.dot(X_st, w))
    for i in range(y_pred.shape[0]):
        if y_pred[i] > 0.5: 
            y_pred[i] = 1
        elif y_pred[i] <= 0.5:
            y_pred[i] = 0
    return y_pred

In [61]:
calc_pred(w, X_st)

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

In [62]:
calc_logloss(y=0, y_pred=1)

11.512925464970229